calculate how many group, the number of vulnerabilities under every group, the total statistics of vuls --- serve for deployment of detection

collect the information about:
- type, location, function, attack_type, trigger_mechanism, cwe/cve

In [1]:
from pathlib import Path
import pandas as pd
import json

In [ ]:
# define process for npm
def load_json_file(json_file: Path) -> json:
    with json_file.open("r") as fr:
        return json.load(fr)

In [ ]:

def npm_json_process(json_file: Path):
    data = load_json_file(json_file)
    type = "npm"
    desc = data['details']
    if "fully compromised" in desc:
        attack_type="data exfiltration/root shell"
        function='install powerful malware'
        cwe="CWE-506"
        trigger_mechanism = "download or running"
        location="entrypoint/download"
    else:
        print(desc)
    hide='N/A'
    return type, location, function, attack_type, trigger_mechanism, hide, cwe


# define process for ruby
def ruby_json_process(json_file: Path):



# define process for rust
def rust_json_process(json_file: Path):
    data = load_json_file(json_file)

    type = "Rust"
    desc = data['details']
    if 'typosquat' in desc and "malware" in desc:
        # malware inside description --- typosquatting
        attack_type="typosquatting/malware"
        function='download payload'
    elif "communicate" in des and "domain" in desc:
        attack_type="C2 communication" 
        function='build communication tunnel with C2'

    if data['id'] == "MAL-2022-1":
        trigger_mechanism="function call/condition/variable configuration"
        location='specific function'

    if data['id'] == "MAL-2023-8429":
        trigger_mechanism="running network connection"
        location='socket or network modules'
    cwe ='N/A'
    hide='N/A'
    return type, location, function, attack_type, trigger_mechanism, hide, cwe

# define process for public articles


In [ ]:
# define process for pypi
def pypi_json_process(json_file: Path):
    data = load_json_file(json_file)
    type = "pypi"
    desc = data["details"]
    if "extension" in desc and "crypto" in desc and "clipboard" in desc:
        attack_type="typosquatting"
        function="access the clipboard, replay crypto wallet address"
        hide="obfuscation"
        cwe="N/A"
        trigger_mechanism="installation"
        location="variable names in Chinese"

    elif "spyware" in desc and "information" in desc:
        


In [ ]:
# iterate the json under per folder
data_folder = Path.cwd().parent.joinpath("malicious")
for number, json_file in enumerate(data_folder.rglob("*.json")):